참고자료 : https://suy379.tistory.com/90?category=937254  
- 이번 챕터에서 배울 수 있는 점
    * 이미지, 동영상 데이터를 불러오는 방법
    * 동영상을 이미지로 나눠서 저장하는 방법
    * 이미지, 동영상 속에서 사람의 얼굴을 검출하는 방법
    * 원본 영상에서 사람 얼굴을 검출하고 짧은 타임랩스 영상 만들기

**OpenCV 라이브러리 : 이미지 및 동영상 데이터 다루는 라이브러리**  
**dlib 라이브러리 : 얼굴 안면 인식을 가능하게 하는 라이브러리**

### **1. 데이터 불러오기**

##### 1.1 이미지 불러오기

**[중요 메서드]**  


    - cv2.imread(path) : path로부터 이미지 파일을 읽어온다.
    - cv2.imshow('file', file) : 파일명을 통해 이미지 파일을 연다
    - cv2.waitKey(숫자) : 숫자에 지정한 밀리초 만큼 이미지를 표시한다. 단, 0으로 지정하면 사용자가 창을 닫아야한다.

In [1]:
import cv2 # 이미지 파일을 다루는 OpenCV 라이브러리 불러오기

In [2]:
# 이미지 파일읽어보기
img = cv2.imread('./data/img/01.jpg')
# 이미지의 길이와 넓이 저장
height, width = img.shape[:2]

print('img width :', width)
print('img height :', height)

img width : 960
img height : 640


In [3]:
# 이미지 확인
cv2.imshow('img', img)
cv2.waitKey(0)

-1

##### 1.2 동영상 불러오기

영상 출처  
mov01 : https://www.youtube.com/watch?v=bp6hhq8DdgU  
mov02 : https://www.youtube.com/watch?v=RICA_Su1blM

**[중요 메서드]**

    - cap = cv2.VideoCapture(path) : path로부터 동영상 파일을 불러와 'cap'으로 저장
    - cap.read() : 영상 정보를 읽어온다.
    - cap.get() : 영상 정보를 읽어오면서 옵션으로 특정 정보만 뽑을 수 있다.
        * cv2.CAP_PROP_FRAME_WIDTH : width 정보
        * cv2.CAP_PROP_FRAME_HEIGHT : height 정보
        * cv2.CAP_PROP_FRAME_COUNT : 영상 총 프레임 수 정보
        * cv2.CAP_PROP_FPS : 영상 fps 값
    - cv2.imshow('file', 'file) : 영상 파일을 연다.
    - cv2.waitKey(숫자) : 숫자에 지정한 밀리초(ms) 만큼 영상을 표시
    - cv2.destroyAllWindows() : 모든 열려있는 창(영상, 이미지)를 닫는다.

In [4]:
# 동영상 불러오기
cap = cv2.VideoCapture('./data/mov/mov01.mp4')
cap

< cv2.VideoCapture 000002634A4628F0>

In [5]:
# 동영상 정보
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)

print('가로 :', str(width))
print('세로 :', str(height))
print('총 프레임 수 :', str(count))
print('FPS :', str(fps))

가로 : 1280.0
세로 : 720.0
총 프레임 수 : 268.0
FPS : 23.976023976023978


In [6]:
# 동영상 - 출력
while(cap.isOpened()) :
    ret, frame = cap.read()
    if ret :
        cv2.imshow('frame', frame)
    # 'q'키를 누르면 영상 종료
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break

cap.release()
cv2.destroyAllWindows()

##### 1.3 동영상을 이미지로 나누고 저장

In [7]:
cap = cv2.VideoCapture('./data/mov/mov01.mp4')
num = 0

while(cap.isOpened()) :
    ret, frame = cap.read()
    if ret :
        cv2.imshow('frame', frame)
        # 이미지의 각 이름을 자동으로 지정
        path ='./data/snapshot/snap01/snapshot_' + str(num) + '.jpg'
        cv2.imwrite(path, frame)
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
    num += 1

cap.release()
cv2.destroyAllWindows()

### **2. 이미지에서 얼굴 인식**

##### 2.1 이미지 속 사람 검출 (HOG-SVMDectector 모델)

**[주요 메서드]**

- hog = cv2.HOGDescriptor() : HOG 객체 만들기
- hog.setSVMDetector(HOGDescriptor_getDefaultPeopleDetector()) : SVMDetector 모델 지정하기
- hogParams = {} : 파라미터 지정하기

- cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) : 흑백으로 변환
- hog.detectMultiScale(객체,hogParams) : 흑백으로 변환한 객체와 파라미터를 input으로 넣고 그 위치정보를 반환
- cv2.rectangel(img, pt1, pt2, color, thickness) : 검출한 사람 주위에 사각형을 그리는 메서드
    * pt1 : 시작점 좌표
    * pt2 : 종료점 좌표
    * color : 사각형 색
    * thickness : 사각형 선 굵기

In [17]:
# 객체 생성
hog = cv2.HOGDescriptor()
# 모델 지정
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
# 파라미터 설정
hogParams = {'winStride' : (8, 8), 'padding' : (32,32), 'scale' : 1.05, 'hitThreshold' : 0}

In [30]:
# 이미지 읽기
img = cv2.imread('./data/img/02.png')
# 흑백변화
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# 사람 검출
human, r = hog.detectMultiScale(gray, **hogParams)

In [31]:
if (len(human) > 0) :
    for (x, y, w, h) in human :
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)
cv2.imshow('img', img)
cv2.waitKey(0)

-1